In [6]:
import networkx as nx
import seaborn as sns 
from IPython.display import IFrame, display # TODO understand how to use visualization without it
import numpy as np
from datetime import datetime
import pandas as pd
import json
from functools import wraps

In [7]:
__TEMPLATE__ = """

<!DOCTYPE html>
<meta charset="cp1251">
<style>

circle {{
    fill: rgba(200, 200, 200, 0.5);
    stroke: rgba(200, 200, 200, 0.5);
    stroke-width: 5.0px;
}}

.circle.source_node {{
    fill: #f3f310;
}}

.circle.nice_node {{
    fill: rgba(0, 256, 0, 0.5);
    stroke: rgba(0, 256, 0, 0.9);
}}

.circle.bad_node {{
    stroke: rgba(256, 0, 0, 0.9);
    fill: rgba(256, 0, 0, 0.5);
}}

.link {{
    fill: none;
    stroke: #666;
    stroke-opacity: 0.2;
}}

#nice_target {{
    fill: green;
}}

.link.nice_target {{
    stroke: green;
}}

#source {{
    fill: yellow;
}}

.link.source {{
    stroke: #f3f310;
}}
                
.link.positive {{
    stroke: green;
}}
                
.link.negative {{
    stroke: red;
}}

#source {{
    fill: orange;
}}

.link.source1 {{
    stroke: orange;
}}

bad_target {{
    fill: red;
}}

.link.bad_target {{
    stroke: red;
}}

text {{
    font: 12px sans-serif;
    pointer-events: none;
    #text-shadow: 0 1px 0 #fff, 1px 0 0 #fff, 0 -1px 0 #fff, -1px 0 0 #fff;
}}
                
#mydiv {{
    position: absolute;
    text-align: center;
    top: 50%;
    left: 50%;
    transform: translate(-50%, -50%);
    background-color: transparent;
}}
                
.wrapper {{
    position: relative;
    width: 100%;
    height: 100%;
}}
                
.close {{
    position: absolute;
    right: 32px;
    top: 32px;
    width: 32px;
    height: 32px;
    opacity: 0.3;
}}
                
.close:hover {{
    opacity: 1;
}}
                
.close:before,
.close:after {{
    position: absolute;
    left: 15px;
    content: " ";
    height: 33px;
    width: 2px;
    background-color: #333;
}}
                
.close:before {{
    transform: rotate(45deg);
}}
                
.close:after {{
    transform: rotate(-45deg);
}}
                           
</style>

<script src="https://d3js.org/d3.v5.min.js"></script>
<script src="https://www.gstatic.com/charts/loader.js"></script>


<div>
  <input type="radio" class="radio" id="radioButton_mean" name="contact" value="mean"><label> Show mean </label>
  <input type="radio" class="radio" id="radioButton_min" name="contact" value="min"><label> Show min </label>
  <input type="radio" class="radio" id="radioButton_max" name="contact" value="max"><label> Show max </label>
</div>

<div class="parent" draggable={{true}}  id="mydiv"></div>
    <div class="wrapper">
        <div class="close" id="close-1"></div>

<table class="columns">
    </table>
<script>

var links = {links};

var node_params = {node_params};

var node_stats = {node_stats};

var nodes = {nodes};

var width = {width},
    height = {height};

var svg = d3.select("body").append("svg")
    .attr("width", width)
    .attr("height", height);
    
google.charts.load("current", {{packages:["corechart"]}});

console.log(svg.selectAll('.radio'));

let defs = svg.append("g").selectAll("marker")
    .data(links)
  .enter().append("marker")
    .attr("id", function(d) {{ return d.source.index + '-' + d.target.index; }})
    .attr("viewBox", "0 -5 10 10")
    .attr("refX", function(d) {{
        if (d.target.name !== d.source.name) {{
            return 7 + d.target.degree; 
        }} else {{
            return 0;
        }}
    }})
    .attr("refY", calcMarkers)
    .attr("markerWidth", 10)
    .attr("markerHeight", 10)
    .attr("markerUnits", "userSpaceOnUse")
    .attr("orient", "auto");

defs.append("path")
    .attr("d", "M0,-5L10,0L0,5");

function calcMarkers(d) {{
    let dist = Math.sqrt((nodes[d.target.index].x - nodes[d.source.index].x) ** 2 + (nodes[d.target.index].y - nodes[d.source.index].y) ** 2);
    if (dist > 0 && dist <= 200){{
        return - Math.sqrt((0.5 - (d.target.degree ) / 2 / dist)) * (d.target.degree) / 2;
    }} else {{
        return 0;
    }}
}}


var path = svg.append("g").selectAll("path")
    .data(links)
  .enter().append("path")
    .attr("class", function(d) {{ return "link " + d.type; }})
    .attr("stroke-width", function(d) {{ return Math.max(d.weight * 20, 1); }})
    .attr("marker-end", function(d) {{ return "url(#" + d.source.index + '-' + d.target.index + ")"; }})
    .attr("id", function(d,i) {{ return "link_"+i; }})
    .attr("parent_pk", function(d) {{ return d.source.pk; }})
    .attr("d", linkArc)
    ;

var edgetext = svg.append("g").selectAll("text")
    .data(links)
   .enter().append("text")
   .append("textPath")
    .attr("xlink:href",function(d,i){{return "#link_"+i;}})
    .style("text-anchor","right")
    .attr("startOffset", "40%")
    ;
    
function update() {{
    d3.selectAll(".radio").each(function(d) {{
        cb = d3.select(this);
        
        if (cb.property("checked")) {{
         radio_text = this.value;
         
            edgetext = edgetext.text(function(d) {{
                if (radio_text == "mean") {{
                    return d.weight_mean;
                }}
                
                 if (radio_text == "min") {{
                    return d.weight_min;
                }}
                
                 if (radio_text == "max") {{
                    return d.weight_max;
                }}
            }})
        }} 
        
        
    }})
}};

d3.selectAll(".radio").on("change", update);

function dragstarted(d) {{
  d3.select(this).raise().classed("active", true);
}}

function dragged(d) {{
  d3.select(this).attr("cx", d.x = d3.event.x).attr("cy", d.y = d3.event.y);
}}

function dragended(d) {{
  d3.select(this).classed("active", false);
  path = path.attr("d", linkArc);
  text = text
        .attr('x', function(d) {{ return d.x; }})
        .attr('y', function(d) {{ return d.y; }})
        ;
  defs = defs.attr("refY", calcMarkers);
  defs.append("path")
    .attr("d", "M0,-5L10,0L0,5");
}};

var circle = svg.append("g").selectAll("circle")
    .data(nodes)
  .enter().append("circle")
    .attr("class", function(d) {{ return "circle " + d.type; }})
    .attr("r", function(d) {{ return d.degree; }})
    .attr('cx', function(d) {{ return d.x; }})
    .attr('cy', function(d) {{ return d.y; }})
    .attr('id', function(d) {{ return d.pk; }})
    .call(d3.drag()
        .on("start", dragstarted)
        .on("drag", dragged)
        .on("end", dragended));

var text = svg.append("g").selectAll("text")
    .data(nodes)
  .enter().append("text")
    .attr('x', function(d) {{ return d.x; }})
    .attr('y', function(d) {{ return d.y; }})
    .text(function(d) {{ return d.name; }});

function linkArc(d) {{
  var dx = nodes[d.target.index].x - nodes[d.source.index].x,
      dy = nodes[d.target.index].y - nodes[d.source.index].y,
      dr = dx * dx + dy * dy;
      dr = Math.sqrt(dr);
      if (dr > 200) {{
        dr *= 5
      }} else {{
        dr *= 5 /*dr /= 2*/
      }};
      if (dr > 0) {{return "M" + nodes[d.source.index].x + "," + nodes[d.source.index].y + "A" + (dr * 1.1) + "," + (dr * 1.1) + " 0 0,1 " + nodes[d.target.index].x + "," + nodes[d.target.index].y;}}
      else {{return "M" + nodes[d.source.index].x + "," + nodes[d.source.index].y + "A" + 20 + "," + 20 + " 0 1,0 " + (nodes[d.target.index].x + 0.1) + "," + (nodes[d.target.index].y + 0.1);}}
}}

function downloadLayout() {{
    var a = document.createElement("a");
    var file = new Blob([JSON.stringify(nodes)], {{type: "text/json;charset=utf-8"}});
    a.href = URL.createObjectURL(file);
    a.download = "node_params.json";
    a.click();
}}

function draw_channels(ind) {{
       
    console.log([['Ветка', 'Процент']].concat(node_stats[ind]));
        
    var data = google.visualization.arrayToDataTable([['Ветка', 'Процент']].concat(node_stats[ind][0]));
 
 
    var options = {{pieResidueSliceLabel: 'Остальное',
                       width:500,
                       height:300,
                       backgroundColor:'transparent'
                       }};

    const parent = document.getElementById("mydiv");
    const new_parent = document.createElement("div");
        
    new_parent.setAttribute("draggable", "true")
    new_parent.setAttribute("id", "mydiv")
    document.body.appendChild(new_parent);
        
    dragElement(new_parent);
        
    const newChild = document.createElement("mydivheader_clone")
    newChild.setAttribute("mydivheader", "mydivheader_clone")
    new_parent.appendChild(newChild);
        
    var chart = new google.visualization.PieChart(newChild);
    chart.draw(data, options)
        
    const close = document.getElementById("close-1");

    close.onclick = e => {{
    const parent = document.getElementById("mydiv");
    parent.remove();
}};
        
}};
      
dragElement(document.getElementById("mydiv"));

function dragElement(elmnt) {{
  var pos1 = 0, pos2 = 0, pos3 = 0, pos4 = 0;
  if (document.getElementById(elmnt.id + "header")) {{
    document.getElementById(elmnt.id + "header").onmousedown = dragMouseDown;
  }} else {{
    elmnt.onmousedown = dragMouseDown;}}
    
function dragMouseDown(e) {{
  e = e || window.event;
  e.preventDefault();
  
  pos3 = e.clientX;
  pos4 = e.clientY;
  
  document.onmouseup = closeDragElement;
  document.onmousemove = elementDrag;
}}

function elementDrag(e) {{
  e = e || window.event;
  e.preventDefault();
  
  pos1 = pos3 - e.clientX;
  pos2 = pos4 - e.clientY;
  pos3 = e.clientX;
  pos4 = e.clientY;
  
  elmnt.style.top = (elmnt.offsetTop - pos2) + "px";
  elmnt.style.left = (elmnt.offsetLeft - pos1) + "px";
}}

function closeDragElement() {{
  document.onmouseup = null;
  document.onmousemove = null;
}}

}}


function set_visibility(next_style, id)
{{
    if (node_stats[id] != undefined) {{
 
svg.selectAll("circle").filter(obj => {{return node_stats[id][1].indexOf(obj.pk) != -1}}).style('visibility',next_style);
path.filter(obj => {{return String(id) == obj.parent_pk}}).style('visibility',next_style);

path.filter(obj => {{ return obj.source.pk == id}}).style('visibility', next_style);

text.filter(obj => {{return node_stats[id][1].indexOf(obj.pk) != -1}}).style('visibility', next_style);

edgetext.filter(obj => {{return obj.source.pk == id}}).style('visibility', next_style);

}}

}}
 
 
svg.selectAll("circle").on("dblclick", function () 
{{

console.log(node_stats[this.id]);

draw_channels(this.id);

curr_style = svg.selectAll("circle").filter(obj => {{return node_stats[this.id][1].indexOf(obj.pk) != -1}})._groups[0][0].style['visibility'];

if (curr_style == 'visible')
next_style = 'hidden';
else
next_style = 'visible';

let to_process = [this.id];
let all_children = [];

while (to_process.length > 0)
{{
    curr_id = to_process.pop();
    if (node_stats[curr_id] != undefined)
    {{
        for (var i = 0; i < node_stats[curr_id][1].length; i++)
        {{
            to_process.push(node_stats[curr_id][1][i]);
        }}
    }}    
    all_children.push(curr_id);
}}

 for (var i = 0; i < all_children.length; i++)
{{
    set_visibility(next_style, all_children[i]);
}}


}});



</script>
"""

In [12]:
# class ___DynamicFigureWrapper__(object):
#     def __init__(self, fig, interactive, width, height):
#         self.fig = fig
#         self.interactive, self.width, self.height = interactive, width, height

#     def get_figure(self):
#         savefig = __SaveFigWrapper__(self.fig, self.interactive, self.width, self.height)
#         return savefig

#     def text(self, x, y, text, *args, **kwargs):
#         parts = self.fig.split('<body>')
#         res = parts[:1] + [f'<p>{text}</p>'] + parts[1:]
#         self.fig = '\n'.join(res)

#     def axis(self):
#         return 4 * [0]

In [18]:
# def _prepare_node_stats(data):
#     result = {}
#     for i, row in data.iterrows():
#         children = data[data['parent_index'] == row['parent_index']]
        
#         children_data = []
#         children_ids = []
        
#         for j, child_row in children.iterrows():
#             children_data.append([child_row['target'], float(str(child_row['weight']).split('%', 1)[0].strip())])
#             children_ids.append(child_row['my_index'])
        
#         result[row['parent_index']] = [children_data, children_ids]
    
#     return result

In [19]:
def graph(data, node_params=None, node2ind=None, thresh=.05, width=500, height=500, interactive=True,
          layout_dump=None, show_percent=True, plot_name=None, **kwargs):
    """
    Plots graph by its edgelist representation

    :param data: graph in edgelist form
    :param node_params: mapping describes which node should be highlighted by target or source type
            Node param should be represented in the following form
            ```{
                    'lost': 'bad_target',
                    'passed': 'nice_target',
                    'onboarding_welcome_screen': 'source',
                }```
            If mapping is not given, it will be constracted from config
    :param thresh: threshold for filtering low frequency edges
    :param width: width of plot
    :param height: height of plot
    :param interactive: if True, then opens graph visualization in Jupyter Notebook IFrame
    :param layout_dump: path to layout dump
    :param show_percent: if True, then all edge weights are converted to percents
    :param kwargs: do nothing, needs for plot.graph usage with other functions
    :return: saves to `experiments_folder` webpage with js graph visualization
    """
    if node_params is None:
        node_params = _prepare_node_params(node_params, data)
    
    node_stats = _prepare_node_stats(data)
    
    res = _make_json_data(data, node_params, node2ind, layout_dump, thresh=thresh,
                          width=width - 100, height=height - 100, **kwargs)
    x = __TEMPLATE__.format(
        width=width,
        height=height,
        links=json.dumps(res.get('links')).encode('latin1').decode('utf-8'),
        node_params=json.dumps(node_params).encode('latin1').decode('utf-8'),
        nodes=json.dumps(res.get('nodes')).encode('latin1').decode('utf-8'),
        node_stats=json.dumps(node_stats).encode('latin1').decode('utf-8'),
        show_percent="1 !== 1" if show_percent else "1 === 1"
    )
    if hasattr(data, 'trajectory'):
        if plot_name is None:
            plot_name = f'{data.trajectory.retention_config["experiments_folder"]}/index_{datetime.now()}'
    else:
        if plot_name is None:
            plot_name = 'index'
    plot_name = plot_name.replace(':', '_').replace('.', '_') + '.html'
    
#     with open('test.html', 'w') as f:
#         f.write(x)
    
    return ___DynamicFigureWrapper__(x, interactive, width, height), plot_name, plot_name



In [8]:
data = pd.read_csv('data_n.csv', index_col=0)
data.head()

,source,target,weight,level,weight_mean,weight_min,weight_max,my_index,parent_index,medium_perc
0,РБ,ВСП (ветка всп),"68% (100%, 68%)",0,NaN,NaN,NaN,19,25,100.0
1,ВСП (ветка всп),Первый платеж (ветка всп),"63% (94%, 94%)",1,23.0,1.0,392.0,15,19,94.0
2,ВСП (ветка всп),Неактивный (ветка всп),"3% (5%, 5%)",1,NaN,NaN,NaN,5,19,5.0
3,ВСП (ветка всп),Закрыт (ветка всп),"1% (1%, 1%)",1,5.0,0.0,60.0,13,19,1.0
5,Первый платеж (ветка всп),Блокировка (ветка всп),"1% (1%, 1%)",2,30.0,0.0,307.0,0,15,1.0


In [33]:
kwargs = {
#                 'edge_col': data.retention.retention_config['index_col'],
                'edge_attributes': '_nunique',
                'norm': True,
                'width':1000, 'height':1000
            }

_node_params = {
                'positive_target_event': 'nice_target',
                'negative_target_event': 'bad_target',
                'source_event': 'source',
            }

# _node_stats = {10: [[['ВСП (ветка всп)', 68.0], ['ДРС', 32.0]], [26, 4]],
#              26: [[['Первый платеж (ветка всп)', 63.0],
#                ['Неактивный (ветка всп)', 3.0],
#                ['Закрыт (ветка всп)', 1.0]],
#               [9, 2, 7]],
#              9: [[['Неактивный 2 месяца (ветка всп)', 0.0],
#                ['Блокировка (ветка всп)', 1.0],
#                ['Активный после платежа (ветка всп)', 61.0],
#                ['Закрытие после платежа (ветка всп)', 2.0]],
#               [1, 19, 8, 12]],
#              19: [[['00.01.1900 (ветка всп)', 0.0],
#                ['Банкротство (ветка всп)', 0.0],
#                ['Налоговая (ветка всп)', 0.0],
#                ['Комплаенс (ветка всп)', 0.0],
#                ['Ликвидация (ветка всп)', 0.0]],
#               [15, 13, 23, 0, 22]],
#              4: [[['Аннулированы', 11.0], ['ВСП (ветка ДРС) (ветка дрс, всп)', 22.0]],
#               [5, 25]],
#              25: [[['Первый платеж (ветка дрс, всп)', 21.0],
#                ['Неактивный (ветка дрс, всп)', 1.0]],
#               [17, 20]],
#              17: [[['Неактивный 2 месяца (ветка дрс, всп)', 0.0],
#                ['Блокировка (ветка дрс, всп)', 0.0],
#                ['Активный после платежа (ветка дрс, всп)', 21.0],
#                ['Закрытие после платежа (ветка дрс, всп)', 0.0]],
#               [3, 11, 18, 21]],
#              11: [[['00.01.1900 (ветка дрс, всп)', 0.0],
#                ['Налоговая (ветка дрс, всп)', 0.0],
#                ['Комплаенс (ветка дрс, всп)', 0.0],
#                ['Ликвидация (ветка дрс, всп)', 0.0]],
#               [16, 6, 14, 24]]
#               }

_node_stats = {}

# links = [{'source': {'pk': 10, 'index': 10, 'name': 'РБ', 'x': 15.0, 'y': 50.0, 'type': 'suit', 'degree': 7}, 'target': {'pk': 26, 'index': 14, 'name': 'ВСП (ветка всп) 68% (100%, 68%)  ', 'x': 52.5, 'y': 195.45454545454544, 'type': 'suit', 'degree': 7}, 'weight': 0.7, 'weight_mean': '', 'weight_min': '', 'weight_max': '', 'type': ''}, {'source': {'pk': 26, 'index': 14, 'name': 'ВСП (ветка всп) 68% (100%, 68%)  ', 'x': 52.5, 'y': 195.45454545454544, 'type': 'suit', 'degree': 7}, 'target': {'pk': 9, 'index': 9, 'name': 'Первый платеж (ветка всп) 63% (94%, 94%)  ', 'x': 165.0, 'y': 268.18181818181813, 'type': 'suit', 'degree': 7}, 'weight': 0.7, 'weight_mean': '23', 'weight_min': '1.0', 'weight_max': '392.0', 'type': ''}, {'source': {'pk': 26, 'index': 14, 'name': 'ВСП (ветка всп) 68% (100%, 68%)  ', 'x': 52.5, 'y': 195.45454545454544, 'type': 'suit', 'degree': 7}, 'target': {'pk': 2, 'index': 4, 'name': 'Неактивный (ветка всп) 3% (5%, 5%)  ', 'x': 165.0, 'y': 340.9090909090909, 'type': 'bad_node', 'degree': 7}, 'weight': 0.7, 'weight_mean': '', 'weight_min': '', 'weight_max': '', 'type': 'bad_target'}, {'source': {'pk': 26, 'index': 14, 'name': 'ВСП (ветка всп) 68% (100%, 68%)  ', 'x': 52.5, 'y': 195.45454545454544, 'type': 'suit', 'degree': 7}, 'target': {'pk': 7, 'index': 7, 'name': 'Закрыт (ветка всп) 1% (1%, 1%)  ', 'x': 165.0, 'y': 413.63636363636357, 'type': 'bad_node', 'degree': 7}, 'weight': 0.7, 'weight_mean': '5', 'weight_min': '0.0', 'weight_max': '60.0', 'type': 'bad_target'}, {'source': {'pk': 9, 'index': 9, 'name': 'Первый платеж (ветка всп) 63% (94%, 94%)  ', 'x': 165.0, 'y': 268.18181818181813, 'type': 'suit', 'degree': 7}, 'target': {'pk': 19, 'index': 2, 'name': 'Блокировка (ветка всп) 1% (1%, 1%)  ', 'x': 427.5, 'y': 340.9090909090909, 'type': 'bad_node', 'degree': 7}, 'weight': 0.7, 'weight_mean': '30', 'weight_min': '0.0', 'weight_max': '307.0', 'type': 'bad_target'}, {'source': {'pk': 9, 'index': 9, 'name': 'Первый платеж (ветка всп) 63% (94%, 94%)  ', 'x': 165.0, 'y': 268.18181818181813, 'type': 'suit', 'degree': 7}, 'target': {'pk': 8, 'index': 8, 'name': 'Активный после платежа (ветка всп) 61% (90%, 96%)  ', 'x': 427.5, 'y': 413.63636363636357, 'type': 'nice_node', 'degree': 7}, 'weight': 0.7, 'weight_mean': '', 'weight_min': '', 'weight_max': '', 'type': 'nice_target'}, {'source': {'pk': 9, 'index': 9, 'name': 'Первый платеж (ветка всп) 63% (94%, 94%)  ', 'x': 165.0, 'y': 268.18181818181813, 'type': 'suit', 'degree': 7}, 'target': {'pk': 12, 'index': 12, 'name': 'Закрытие после платежа (ветка всп) 2% (2%, 3%)  ', 'x': 427.5, 'y': 486.3636363636363, 'type': 'bad_node', 'degree': 7}, 'weight': 0.7, 'weight_mean': '29', 'weight_min': '0.0', 'weight_max': '332.0', 'type': 'bad_target'}, {'source': {'pk': 10, 'index': 10, 'name': 'РБ', 'x': 15.0, 'y': 50.0, 'type': 'suit', 'degree': 7}, 'target': {'pk': 4, 'index': 5, 'name': 'ДРС 32% (100%, 32%)  ', 'x': 52.5, 'y': 340.9090909090909, 'type': 'suit', 'degree': 7}, 'weight': 0.7, 'weight_mean': '', 'weight_min': '', 'weight_max': '', 'type': ''}, {'source': {'pk': 4, 'index': 5, 'name': 'ДРС 32% (100%, 32%)  ', 'x': 52.5, 'y': 340.9090909090909, 'type': 'suit', 'degree': 7}, 'target': {'pk': 5, 'index': 6, 'name': 'Аннулированы 11% (33%, 33%)  ', 'x': 165.0, 'y': 413.63636363636357, 'type': 'bad_node', 'degree': 7}, 'weight': 0.7, 'weight_mean': '', 'weight_min': '', 'weight_max': '', 'type': 'bad_target'}, {'source': {'pk': 4, 'index': 5, 'name': 'ДРС 32% (100%, 32%)  ', 'x': 52.5, 'y': 340.9090909090909, 'type': 'suit', 'degree': 7}, 'target': {'pk': 25, 'index': 13, 'name': 'ВСП (ветка ДРС) (ветка дрс, всп) 22% (67%, 67%)  ', 'x': 165.0, 'y': 486.3636363636363, 'type': 'suit', 'degree': 7}, 'weight': 0.7, 'weight_mean': '5', 'weight_min': '0.0', 'weight_max': '220.0', 'type': ''}, {'source': {'pk': 25, 'index': 13, 'name': 'ВСП (ветка ДРС) (ветка дрс, всп) 22% (67%, 67%)  ', 'x': 165.0, 'y': 486.3636363636363, 'type': 'suit', 'degree': 7}, 'target': {'pk': 17, 'index': 0, 'name': 'Первый платеж (ветка дрс, всп) 21% (65%, 96%)  ', 'x': 427.5, 'y': 559.090909090909, 'type': 'suit', 'degree': 7}, 'weight': 0.7, 'weight_mean': '25', 'weight_min': '1.0', 'weight_max': '392.0', 'type': ''}, {'source': {'pk': 25, 'index': 13, 'name': 'ВСП (ветка ДРС) (ветка дрс, всп) 22% (67%, 67%)  ', 'x': 165.0, 'y': 486.3636363636363, 'type': 'suit', 'degree': 7}, 'target': {'pk': 20, 'index': 3, 'name': 'Неактивный (ветка дрс, всп) 1% (2%, 4%)  ', 'x': 427.5, 'y': 631.8181818181818, 'type': 'bad_node', 'degree': 7}, 'weight': 0.7, 'weight_mean': '', 'weight_min': '', 'weight_max': '', 'type': 'bad_target'}, {'source': {'pk': 17, 'index': 0, 'name': 'Первый платеж (ветка дрс, всп) 21% (65%, 96%)  ', 'x': 427.5, 'y': 559.090909090909, 'type': 'suit', 'degree': 7}, 'target': {'pk': 11, 'index': 11, 'name': 'Блокировка (ветка дрс, всп) 0% (1%, 1%)  ', 'x': 615.0, 'y': 704.5454545454545, 'type': 'bad_node', 'degree': 7}, 'weight': 0.7, 'weight_mean': '32', 'weight_min': '0.0', 'weight_max': '288.0', 'type': 'bad_target'}, {'source': {'pk': 17, 'index': 0, 'name': 'Первый платеж (ветка дрс, всп) 21% (65%, 96%)  ', 'x': 427.5, 'y': 559.090909090909, 'type': 'suit', 'degree': 7}, 'target': {'pk': 18, 'index': 1, 'name': 'Активный после платежа (ветка дрс, всп) 21% (64%, 99%)  ', 'x': 615.0, 'y': 850.0, 'type': 'nice_node', 'degree': 7}, 'weight': 0.7, 'weight_mean': '', 'weight_min': '', 'weight_max': '', 'type': 'nice_target'}]
links=[]
nodes = [{'pk': 17, 'index': 0, 'name': 'Первый платеж (ветка дрс, всп) 21% (65%, 96%)  ', 'x': 427.5, 'y': 559.090909090909, 'type': 'suit', 'degree': 7}, {'pk': 18, 'index': 1, 'name': 'Активный после платежа (ветка дрс, всп) 21% (64%, 99%)  ', 'x': 615.0, 'y': 850.0, 'type': 'nice_node', 'degree': 7}, {'pk': 19, 'index': 2, 'name': 'Блокировка (ветка всп) 1% (1%, 1%)  ', 'x': 427.5, 'y': 340.9090909090909, 'type': 'bad_node', 'degree': 7}, {'pk': 20, 'index': 3, 'name': 'Неактивный (ветка дрс, всп) 1% (2%, 4%)  ', 'x': 427.5, 'y': 631.8181818181818, 'type': 'bad_node', 'degree': 7}, {'pk': 2, 'index': 4, 'name': 'Неактивный (ветка всп) 3% (5%, 5%)  ', 'x': 165.0, 'y': 340.9090909090909, 'type': 'bad_node', 'degree': 7}, {'pk': 4, 'index': 5, 'name': 'ДРС 32% (100%, 32%)  ', 'x': 52.5, 'y': 340.9090909090909, 'type': 'suit', 'degree': 7}, {'pk': 5, 'index': 6, 'name': 'Аннулированы 11% (33%, 33%)  ', 'x': 165.0, 'y': 413.63636363636357, 'type': 'bad_node', 'degree': 7}, {'pk': 7, 'index': 7, 'name': 'Закрыт (ветка всп) 1% (1%, 1%)  ', 'x': 165.0, 'y': 413.63636363636357, 'type': 'bad_node', 'degree': 7}, {'pk': 8, 'index': 8, 'name': 'Активный после платежа (ветка всп) 61% (90%, 96%)  ', 'x': 427.5, 'y': 413.63636363636357, 'type': 'nice_node', 'degree': 7}, {'pk': 9, 'index': 9, 'name': 'Первый платеж (ветка всп) 63% (94%, 94%)  ', 'x': 165.0, 'y': 268.18181818181813, 'type': 'suit', 'degree': 7}, {'pk': 10, 'index': 10, 'name': 'РБ', 'x': 15.0, 'y': 50.0, 'type': 'suit', 'degree': 7}, {'pk': 11, 'index': 11, 'name': 'Блокировка (ветка дрс, всп) 0% (1%, 1%)  ', 'x': 615.0, 'y': 704.5454545454545, 'type': 'bad_node', 'degree': 7}, {'pk': 12, 'index': 12, 'name': 'Закрытие после платежа (ветка всп) 2% (2%, 3%)  ', 'x': 427.5, 'y': 486.3636363636363, 'type': 'bad_node', 'degree': 7}, {'pk': 25, 'index': 13, 'name': 'ВСП (ветка ДРС) (ветка дрс, всп) 22% (67%, 67%)  ', 'x': 165.0, 'y': 486.3636363636363, 'type': 'suit', 'degree': 7}, {'pk': 26, 'index': 14, 'name': 'ВСП (ветка всп) 68% (100%, 68%)  ', 'x': 52.5, 'y': 195.45454545454544, 'type': 'suit', 'degree': 7}]

node2ind = { name: i for i, name in enumerate(set(data['source'].values) | set(data['target'].values)) }

In [43]:
nodes = [{'pk': 1, 'index': 0, 'name': '1', 'x': 427.5, 'y': 559.090909090909, 'type': 'suit', 'degree': 1},
        {'pk': 2, 'index': 1, 'name': '2', 'x': 527.5, 'y': 559.090909090909, 'type': 'suit', 'degree': 1}]

In [48]:
links = [{'source': nodes[0], 'target': nodes[1]}]

In [ ]:
node_stats = [{0:]

In [49]:
x = __TEMPLATE__.format(
        width=1000,
        height=1000,
        links=json.dumps(links).encode('latin1').decode('utf-8'),
        node_params=json.dumps(_node_params).encode('latin1').decode('utf-8'),
        nodes=json.dumps(nodes).encode('latin1').decode('utf-8'),
        node_stats=json.dumps(node_stats).encode('latin1').decode('utf-8'),
        show_percent="1 !== 1"
        )

In [50]:
with open('test1.html', 'w') as f:
        f.write(x)

In [51]:
display(IFrame('test1.html', width=1100, height=1200))